## BCycle Data 
Notebook to predict the number of bikes leaving from each station per hour

## Getting the data from Blob Storage
Our data is stored inside of Azure Blob Storage. Following is the code to get it to your notebook.

In [1]:
# If you are running this code on your own computer, uncomment the following line.
# !pip install azure

In [31]:
from azure.storage.blob import BlockBlobService
import os

In [32]:
# You should only run this code once.
!mkdir bcycle

mkdir: cannot create directory ‘bcycle’: File exists


In [33]:
local_path=os.getcwd() + "/bcycle" # Where the data will be stored.
blob_account_name = "mldsdatahack2019diag" # Don't edit!
blob_account_key = "JsauBssnY92CeD3MgI2SWhkQ16JioJCRWVW8NzKtcWckI+DaNNbCmpmMAVq27GD91mhgH+oHPx+QbIKUCow5gA=="  # Don't edit!

mycontainer = "datahackdata2019"       # Don't edit!

In [34]:
import azure
from azure.storage.blob import BlockBlobService

# Create a client to get data
blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)
# Get the data
blob_service.get_blob_to_path("datahackdata2019", "BCycleAustin.csv", os.path.join(local_path, "BCycleAustin.csv"))

## Load Data

In [35]:
# Yo should see BCycleAustin.csv
!ls bcycle

BCycleAustin.csv


In [36]:
import pandas as pd
import matplotlib.pyplot as plt

In [37]:
# Read in data
df = pd.read_csv('bcycle/BCycleAustin.csv')

In [38]:
# See the first few rows.
df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Checkout DateTime,Checkout Latitude,Checkout Longitude,Return Latitude,Return Longitude
0,9900285854,Annual (San Antonio B-cycle),207.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,76,2014-10-26 13:12:00,30.27,-97.75,30.26,-97.74
1,9900288020,24-Hour Kiosk (Austin B-cycle),190.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,12,2014-10-30 19:12:00,30.27,-97.75,30.26,-97.74
2,9900289011,Local365,190.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,9,2015-10-01 19:12:27,30.27,-97.75,30.26,-97.74
3,9900292552,Walk Up,354.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,14,2015-10-04 13:12:41,30.27,-97.75,30.26,-97.74
4,9900293709,Local365,20.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,11,2015-10-04 13:12:56,30.27,-97.75,30.26,-97.74


In [39]:
df.info(())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027711 entries, 0 to 1027710
Data columns (total 13 columns):
Trip ID                  1027711 non-null int64
Membership Type          1027711 non-null object
Bicycle ID               1027711 non-null float64
Checkout Kiosk ID        1027711 non-null float64
Checkout Kiosk           1027711 non-null object
Return Kiosk ID          1027711 non-null float64
Return Kiosk             1027711 non-null object
Trip Duration Minutes    1027711 non-null int64
Checkout DateTime        1027711 non-null object
Checkout Latitude        1027711 non-null float64
Checkout Longitude       1027711 non-null float64
Return Latitude          1027711 non-null float64
Return Longitude         1027711 non-null float64
dtypes: float64(7), int64(2), object(4)
memory usage: 101.9+ MB


##  Get the hour when a bike left

In [40]:
# Convert the checkout datetime from string to an actual date-time object.
df['Checkout DateTime'] = pd.to_datetime(df['Checkout DateTime'])

In [41]:
# Checkout datetime is now a datetime
df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Checkout DateTime,Checkout Latitude,Checkout Longitude,Return Latitude,Return Longitude
0,9900285854,Annual (San Antonio B-cycle),207.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,76,2014-10-26 13:12:00,30.27,-97.75,30.26,-97.74
1,9900288020,24-Hour Kiosk (Austin B-cycle),190.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,12,2014-10-30 19:12:00,30.27,-97.75,30.26,-97.74
2,9900289011,Local365,190.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,9,2015-10-01 19:12:27,30.27,-97.75,30.26,-97.74
3,9900292552,Walk Up,354.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,14,2015-10-04 13:12:41,30.27,-97.75,30.26,-97.74
4,9900293709,Local365,20.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,11,2015-10-04 13:12:56,30.27,-97.75,30.26,-97.74


In [42]:
# Get the checkout year, month, day, hour
df['Checkout Hour'] = df['Checkout DateTime'].apply(lambda x : x.hour)
df['Checkout Day'] = df['Checkout DateTime'].apply(lambda x: x.day)
df['Checkout Month'] = df['Checkout DateTime'].apply(lambda x: x.month)
df['Checkout Year'] = df['Checkout DateTime'].apply(lambda x: x.year)


In [43]:
df.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Checkout DateTime,Checkout Latitude,Checkout Longitude,Return Latitude,Return Longitude,Checkout Hour,Checkout Day,Checkout Month,Checkout Year
0,9900285854,Annual (San Antonio B-cycle),207.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,76,2014-10-26 13:12:00,30.27,-97.75,30.26,-97.74,13,26,10,2014
1,9900288020,24-Hour Kiosk (Austin B-cycle),190.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,12,2014-10-30 19:12:00,30.27,-97.75,30.26,-97.74,19,30,10,2014
2,9900289011,Local365,190.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,9,2015-10-01 19:12:27,30.27,-97.75,30.26,-97.74,19,1,10,2015
3,9900292552,Walk Up,354.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,14,2015-10-04 13:12:41,30.27,-97.75,30.26,-97.74,13,4,10,2015
4,9900293709,Local365,20.00,2537.00,West & 6th St.,2707.00,Rainey St @ Cummings,11,2015-10-04 13:12:56,30.27,-97.75,30.26,-97.74,13,4,10,2015


In [44]:
bikes_left = df.groupby(['Checkout Kiosk ID','Checkout Year','Checkout Month','Checkout Day','Checkout Hour'],sort=False).size().reset_index(name='Count')
bikes_left = bikes_left.drop(columns=['Checkout Year','Checkout Month','Checkout Day'])
bikes_left = bikes_left.dropna()

bikes_arriving = df.groupby(['Return Kiosk ID','Checkout Year','Checkout Month','Checkout Day','Checkout Hour'],sort=False).size().reset_index(name='Count')
bikes_arriving = bikes_arriving.drop(columns=['Checkout Year','Checkout Month','Checkout Day'])
bikes_arriving = bikes_arriving.dropna()

In [45]:
bikes_left.head()

,Checkout Kiosk ID,Checkout Hour,Count
0,2537.00,13,3
1,2537.00,19,3
2,2537.00,19,7
3,2537.00,13,3
4,2537.00,17,1


In [46]:
bikes_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420261 entries, 0 to 420260
Data columns (total 3 columns):
Checkout Kiosk ID    420261 non-null float64
Checkout Hour        420261 non-null int64
Count                420261 non-null int64
dtypes: float64(1), int64(2)
memory usage: 12.8 MB


In [47]:
bikes_left["Checkout Kiosk ID"] = bikes_left["Checkout Kiosk ID"].astype('category')
bikes_arriving["Return Kiosk ID"] = bikes_arriving["Return Kiosk ID"].astype('category')
bikes_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420261 entries, 0 to 420260
Data columns (total 3 columns):
Checkout Kiosk ID    420261 non-null category
Checkout Hour        420261 non-null int64
Count                420261 non-null int64
dtypes: category(1), int64(2)
memory usage: 10.0 MB


In [48]:
# Convert categorical variable into dummy/indicator variables
bikes_left = pd.get_dummies(bikes_left)
bikes_arriving = pd.get_dummies(bikes_arriving)

In [49]:
bikes_left.head()

,Checkout Hour,Count,Checkout Kiosk ID_1001.0,Checkout Kiosk ID_1002.0,Checkout Kiosk ID_1003.0,Checkout Kiosk ID_1005.0,Checkout Kiosk ID_1006.0,Checkout Kiosk ID_1007.0,Checkout Kiosk ID_1008.0,Checkout Kiosk ID_2494.0,...,Checkout Kiosk ID_4051.0,Checkout Kiosk ID_4052.0,Checkout Kiosk ID_4054.0,Checkout Kiosk ID_4055.0,Checkout Kiosk ID_4057.0,Checkout Kiosk ID_4058.0,Checkout Kiosk ID_4059.0,Checkout Kiosk ID_4060.0,Checkout Kiosk ID_4061.0,Checkout Kiosk ID_4062.0
0,13,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
len(bikes_left)

420261

## Get the training and testing data

In [51]:
X = bikes_left.drop(columns=['Count'])
Y = bikes_left.loc[:,['Count']]

In [52]:
X.head()

,Checkout Hour,Checkout Kiosk ID_1001.0,Checkout Kiosk ID_1002.0,Checkout Kiosk ID_1003.0,Checkout Kiosk ID_1005.0,Checkout Kiosk ID_1006.0,Checkout Kiosk ID_1007.0,Checkout Kiosk ID_1008.0,Checkout Kiosk ID_2494.0,Checkout Kiosk ID_2495.0,...,Checkout Kiosk ID_4051.0,Checkout Kiosk ID_4052.0,Checkout Kiosk ID_4054.0,Checkout Kiosk ID_4055.0,Checkout Kiosk ID_4057.0,Checkout Kiosk ID_4058.0,Checkout Kiosk ID_4059.0,Checkout Kiosk ID_4060.0,Checkout Kiosk ID_4061.0,Checkout Kiosk ID_4062.0
0,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
len(Y)

420261

In [54]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=223)
# flatten y_train to 1d array
y_train.values.flatten()

array([1, 1, 1, ..., 4, 1, 1])

## Model Training Using Azure AutoML

In [55]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
import logging
import os

In [82]:
#ws = Workspace.create("datahack-workspace", subscription_id="a0005a02-eceb-4ae6-866c-c8fcbec43ea7",resource_group="Datahack2019",location='eastus')
experiment_name = 'automated-ml-regression'
# project folder
project_folder = './automated-ml-regression'

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

Deploying StorageAccount with name datahackstorageicdblhnd.
Deployed StorageAccount with name datahackstorageicdblhnd.
Deploying ContainerRegistry with name datahackacrhxfpybak.
Deployed ContainerRegistry with name datahackacrhxfpybak.
Deploying AppInsights with name datahackinsightsoertmucb.
Deployed AppInsights with name datahackinsightsoertmucb.
Deploying KeyVault with name datahackkeyvaultusjilajl.
Deployed KeyVault with name datahackkeyvaultusjilajl.
Deploying Workspace with name datahack-workspace.
Deployed Workspace with name datahack-workspace.


,
Location,eastus
Project Directory,./automated-ml-regression
Resource Group,Datahack2019
SDK version,1.0.23
Subscription ID,a0005a02-eceb-4ae6-866c-c8fcbec43ea7
Workspace,datahack-workspace


In [83]:
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 30,
    "primary_metric" : 'spearman_correlation',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

In [84]:
from azureml.train.automl import AutoMLConfig

# local compute
automated_ml_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automated_ml_errors.log',
                             path = project_folder,
                             X = x_train.values,
                             y = y_train.values.flatten(),
                             **automl_settings)

In [85]:
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_97361757-e6eb-44f9-ab26-5b836d1135c5
****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler RandomForest                      0:00:22       0.1535    0.1535
         1   StandardScalerWrapper DecisionTree             0:00:15       0.2948    0.2948
         2   StandardScalerWrapper LightGBM                 0:00:14       0.1430    0.2948
         3   StandardScalerWrapper LightGBM                 0:00:22       0.0421    0.2948
 

In [86]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automated-ml-regression,
Id: AutoML_97361757-e6eb-44f9-ab26-5b836d1135c5_28,
Type: None,
Status: Completed)
RegressionPipeline(mean=-0.0013557359186552789,
          pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(is_onnx_compatible=None, logger=None, task=None)), ('StandardScalerWrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x7f7c6661e400>), ('XGBoostRegressor', <automl.client.core.common.model_wrappers.XGBoostRegressor object at 0x7f7c6661e5f8>)]),
          stddev=5.0279184538063175)


In [88]:
y_predict = fitted_model.predict(x_test.values)

In [89]:
import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE: %f" % (rmse))

RMSE: 2.283747


# one more try using xgboost

In [72]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [73]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [75]:
xg_reg.fit(x_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [76]:
preds = xg_reg.predict(x_test)

In [78]:
import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 2.624127


In [79]:
preds

array([1.4522419, 1.7539995, 1.4522419, ..., 1.7928886, 1.624563 ,
       1.7928886], dtype=float32)